In [ ]:
#| include: false
import pandas as pd
from nbdev.showdoc import *

This example notebook covers ways to generate synthetic data using `numerblox` components. Synthetic data can be a great way to improve performance simply by having more data to train. We will both cover ways to generate synthetic target variables and features.

## 0. Download and load

In [ ]:
from numerblox.download import NumeraiClassicDownloader
from numerblox.numerframe import create_numerframe, NumerFrame

In [ ]:
dl = NumeraiClassicDownloader(directory_path="synth_test")
dl.download_training_data(version=3)

In [ ]:
dataf = create_numerframe("synth_test/numerai_training_data.parquet")

In [ ]:
dataf.head(2)

## 1. Synthetic target (Bayesian GMM)

First we will tackle the problem of creating a synthetic target column to improve model performance. `BayesianGMMTargetProcessor` allows you to generate a new target variable based on a given target. The preprocessor sample the target from a [Bayesian Gaussian Mixture model](https://scikit-learn.org/stable/modules/generated/sklearn.mixture.BayesianGaussianMixture.html) which is fitted on coefficients from a [regularized linear model (Ridge regression)](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html).

This implementation is based on a [Github Gist by Michael Oliver (mdo)](https://gist.github.com/the-moliver/dcdd2862dc2c78dda600f1b449071c93).

In [ ]:
from numerblox.preprocessing import BayesianGMMTargetProcessor

In [ ]:
show_doc(BayesianGMMTargetProcessor)

In [ ]:
dataf.head()

In [ ]:
bgmm = BayesianGMMTargetProcessor(target_col="target_nomi_20")
test_columns = ['era', 'data_type', 'feature_dichasial_hammier_spawner',
                'feature_rheumy_epistemic_prancer', 'target',
                'target_nomi_20', 'target_paul_20']
sample_dataf = NumerFrame(dataf[test_columns].sample(100).fillna(0.5))
fake_dataf = bgmm(sample_dataf)

In [ ]:
sample_dataf.head()

The new target will be suffixed by `_fake` to distinguish it from the original targets.

In [ ]:
fake_dataf.get_target_data.head(2)

Note that you can easily generate multiple fake targets in a loop.

In [ ]:
for target_col in sample_dataf.target_cols:
    bgmm = BayesianGMMTargetProcessor(target_col=target_col)
    sample_dataf = bgmm(sample_dataf)
sample_dataf.get_target_data.head(2)

## 2. DeepDreamGenerator

In [ ]:
from numerblox.preprocessing import DeepDreamGenerator

In [ ]:
show_doc(DeepDreamGenerator)

### 2.1. Simple data generation example

For our example we will use the model open sourced by [nemethpeti](https://github.com/nemethpeti) which you can download [here](https://github.com/nemethpeti/numerai/blob/main/DeepDream/model.h5). This model works on the v3 medium feature set. We therefore use v3 data in this example. The v3 medium feature set can be easily retrieved using `NumeraiClassicDownloader`.

In [ ]:
#| output: false
feature_set = dl.get_classic_features(filename="v3/features.json")
feature_names = feature_set['feature_sets']['medium']

In [ ]:
ddg = DeepDreamGenerator(model_path="../test_assets/deepdream_model.h5",
                         feature_names=feature_names)

2022-05-05 12:40:43.915881: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Let's try to generate features from a small subset of 100 rows.

In [ ]:
sample_dataf_2 = NumerFrame(dataf.sample(100))

In [ ]:
dreamed_dataf = ddg.transform(sample_dataf_2)

Deepdreaming Synthetic Batches:   0%|          | 0/1 [00:00<?, ?it/s]

✅ Finished step DeepDreamGenerator. Output shape=(199, 1073). Time taken for step: 
0:00:00.273758. ✅

The new dreamed `NumerFrame` consists of the original data and 100 new additional rows. Note that targets are the same.

Also, `era`, `data_type` and any other columns besides features and targets will be `NaN`s.

In [ ]:
print(dreamed_dataf.shape)
dreamed_dataf.tail()

(199, 1073)


,era,data_type,feature_dichasial_hammier_spawner,feature_rheumy_epistemic_prancer,feature_pert_performative_hormuz,feature_hillier_unpitied_theobromine,feature_perigean_bewitching_thruster,feature_renegade_undomestic_milord,feature_koranic_rude_corf,feature_demisable_expiring_millepede,...,target_paul_20,target_paul_60,target_george_20,target_george_60,target_william_20,target_william_60,target_arthur_20,target_arthur_60,target_thomas_20,target_thomas_60
94,NaN,NaN,0.000000,0.000000,0.926837,0.701231,0.988952,0.195643,0.780995,0.321617,...,0.50,0.00,0.50,0.25,0.500000,0.166667,0.500000,0.166667,0.500000,0.166667
95,NaN,NaN,0.480184,0.427976,0.223835,0.183721,0.761205,0.395356,0.238707,0.997490,...,1.00,0.75,1.00,0.75,1.000000,0.833333,0.833333,0.833333,0.833333,0.666667
96,NaN,NaN,0.970524,0.246957,0.463982,0.280058,1.000000,0.766258,0.239566,0.723194,...,0.75,0.75,0.50,0.75,0.666667,0.666667,0.666667,0.666667,0.500000,0.666667
97,NaN,NaN,0.010460,1.000000,0.247279,0.204357,0.757548,0.702845,1.000000,0.468156,...,0.25,0.25,0.25,0.25,0.333333,0.333333,0.333333,0.333333,0.333333,0.333333
98,NaN,NaN,0.736778,0.750979,0.007622,0.220263,0.268438,0.741399,0.044169,0.976056,...,0.50,0.50,0.25,0.50,0.500000,0.500000,0.333333,0.333333,0.500000,0.500000


To only get new synthetic data use `.get_synthetic_batch`.


In [ ]:
synth_dataf = ddg.get_synthetic_batch(sample_dataf_2)

Deepdreaming Synthetic Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
print(synth_dataf.shape)
synth_dataf.head()

(99, 441)


,feature_abstersive_emotional_misinterpreter,feature_accessorial_aroused_crochet,feature_acerb_venusian_piety,feature_affricative_bromic_raftsman,feature_agile_unrespited_gaucho,feature_agronomic_cryptal_advisor,feature_alkaline_pistachio_sunstone,feature_altern_unnoticed_impregnation,feature_ambisexual_boiled_blunderer,feature_amoebaean_wolfish_heeler,...,target_paul_20,target_paul_60,target_george_20,target_george_60,target_william_20,target_william_60,target_arthur_20,target_arthur_60,target_thomas_20,target_thomas_60
0,0.000000,0.517510,0.590794,0.000000,1.000000,0.108258,0.042904,0.531423,0.994126,0.322113,...,0.50,0.50,0.75,0.75,0.666667,0.500000,0.500000,0.500000,0.666667,0.666667
1,0.007362,0.018246,0.816708,0.000000,0.050006,0.079645,0.055798,0.000000,0.000000,0.000000,...,0.50,0.50,0.25,0.25,0.500000,0.500000,0.333333,0.333333,0.333333,0.333333
2,1.000000,0.731188,0.384121,0.424877,1.000000,0.120615,0.012530,0.010341,0.433381,0.050186,...,0.50,0.75,0.75,0.50,0.666667,0.666667,0.666667,0.666667,0.500000,0.666667
3,0.009710,0.026749,0.858364,0.420578,0.720584,0.041099,0.000000,0.010843,0.733748,0.477730,...,0.75,0.50,0.50,0.50,0.500000,0.500000,0.500000,0.333333,0.500000,0.500000
4,0.458052,0.623646,0.371411,0.000000,0.000000,0.740971,1.000000,0.746225,1.000000,0.000000,...,0.75,0.75,0.75,0.75,0.666667,1.000000,0.666667,0.833333,0.666667,0.666667


### 2.2. Improve Numerai performance with synthetic data.

Now we will demonstrate that `DeepDreamGenerator` will actually lead to better validation performance when the synthetic data is mixed with the original Numerai data. We hope to see more experiments by the Numerai community with v4 data, different feature sets, etc. Hope this section makes it easy for participants to get started and to setup up your own experiments.

In [ ]:
import numpy as np
import lightgbm as lgb

from numerblox.preprocessing import FeatureSelectionPreProcessor
from numerblox.evaluation import NumeraiClassicEvaluator

In [ ]:
dataf = create_numerframe("synth_test/numerai_training_data.parquet")
val_dataf = create_numerframe("deepdream_eval_test/numerai_validation_data.parquet")

In [ ]:
ddg = DeepDreamGenerator("../test_assets/deepdream_model.h5", batch_size=200_000)

We will train a LightGBM model and evaluate the most common metrics for both original data and original data + ~5% synthetic data (500000 rows).

In [ ]:
def train_and_evaluate(dataf: NumerFrame, val_dataf: NumerFrame, feature_names: list):
    """ Train LightGBM model with proper parameters and evaluate. """
    X_train, y_train = dataf.get_feature_target_pair(multi_target=False)
    X_val, y_val = val_dataf.get_feature_target_pair(multi_target=False)
    lgb_model = lgb.LGBMRegressor(random_state=42, n_estimators=2000, max_depth=5,
                                  learning_rate=.01, colsample_bytree=.1)
    X_train = NumerFrame(X_train[feature_names])
    X_val = NumerFrame(X_val[feature_names])
    lgb_model.fit(X_train, y_train)

    X_val['prediction'] = lgb_model.predict(X_val)
    X_val['era'] = val_dataf['era']
    X_val['target'] = y_val
    X_val['random_pred'] = np.random.uniform(len(y_val))

    evaluator = NumeraiClassicEvaluator(fast_mode=True)
    results = evaluator.full_evaluation(X_val, example_col="random_pred",
                                        target_col='target', pred_cols=['prediction'])
    return results

As reference, We train a LightGBM model on the medium feature set with no added synthetic data.

In [ ]:
results = train_and_evaluate(dataf, val_dataf, feature_names=feature_names)

Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

2022-05-05 12:47:03,481 INFO numexpr.utils: Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2022-05-05 12:47:03,483 INFO numexpr.utils: NumExpr defaulting to 8 threads.


`FeatureSelectionPreProcessor` selects features and keep all other columns, like target, era and data type. We will use this to filter on the medium feature set. Then we randomly take 500000 rows to generate synthetic data on.

In [ ]:
sample_dataf_dream = FeatureSelectionPreProcessor(feature_cols=feature_names)(dataf)
sample_dataf_dream = sample_dataf_dream.sample(500_000, random_state=42)

✅ Finished step FeatureSelectionPreProcessor. Output shape=(2412105, 443). Time taken for 
step: 0:00:03.329357. ✅

A simple call to the `DeepDreamGenerator` will generate a full new dataset from the input DataFrame. In this case, we generate 500000 new rows and add it to our full Numerai v3 dataset.

In [ ]:
sample_dream_dataf = ddg.get_synthetic_batch(sample_dataf_dream)
dream_dataf = pd.concat([dataf, sample_dream_dataf])

Deepdreaming Synthetic Batches:   0%|          | 0/3 [00:00<?, ?it/s]

Now let's train on the full dataset again with 500000 additional synthetic rows and compare results.

In [ ]:
dream_results = train_and_evaluate(dream_dataf, val_dataf,
                                   feature_names=feature_names)

Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
results

,target,mean,std,sharpe,max_drawdown,apy,mmc_mean,mmc_std,mmc_sharpe,corr_with_example_preds
prediction,target,0.020226,0.032328,0.625658,-0.192148,155.268204,0.015526,0.024817,0.62565,-3.130347e-19


In [ ]:
dream_results

,target,mean,std,sharpe,max_drawdown,apy,mmc_mean,mmc_std,mmc_sharpe,corr_with_example_preds
prediction,target,0.020592,0.030987,0.664546,-0.182301,160.191965,0.015807,0.023788,0.664535,-3.072266e-18


Note that with the added synthetic data the mean correlation improves, along with a higher Sharpe and lower max. drawdown. There seems to be a sweet spot when mixing in synthetic data. Add too little and no significant improvement occurs, but adding too much synthetic data can also hurt performance. You can play with this for yourself. Also try to adjust the `steps` and `step_size` parameters in `DeepDreamGenerator`.

## 3. UMAPFeatureGenerator

UMAP is a feature reduction technique that can be used to generate synthetic features. In other words, we create new representations of the existing features and add them to our dataset.

We will perform UMAP on the training and validation data combined. Note that the data created with `DeepDreamGenerator` is included in this dataset. Then, once again we train a model on it and evaluate results.

In [ ]:
from numerblox.preprocessing import UMAPFeatureGenerator

`n_components` denotes the amount of additional features we are generating.

In [ ]:
n_components = 3
umap_gen = UMAPFeatureGenerator(n_components=n_components, n_neighbors=9)

In [ ]:
test_data = create_numerframe("../test_assets/mini_numerai_version_2_data.parquet")

In [ ]:
test_data = umap_gen(test_data)

OMP: Info #271: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


The new features follow the naming convention `f"feature_umap_{i}"`. All new components are scaled between 0 and 1.

In [ ]:
umap_features = [f"feature_umap_{i}" for i in range(n_components)]
test_data[umap_features].head(3)

,feature_umap_0,feature_umap_1,feature_umap_2
id,,,
n559bd06a8861222,0.887313,0.365509,1.000000
n9d39dea58c9e3cf,1.000000,0.779677,0.732083
nb64f06d3a9fc9f1,0.879256,0.073302,0.174605


Contrast this with the deep dream results.

After you're done all the downloaded files can be cleaned up with `.remove_base_directory()`.

In [ ]:
# Clean up environment
dl.remove_base_directory()

⚠ Deleting directory for 'NumeraiClassicDownloader' ⚠
Path: 
'/Users/clepelaars/Desktop/crowdcent/repositories/numerai-blocks/nbs/edu_nbs/synth_test'